In [45]:
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
import pdfplumber
import re
import os
from dotenv import load_dotenv

# 환경변수 불러오기
load_dotenv()

True

#### File 불러오기

In [46]:
# 파일 경로 지정
file_path = 'KIET_경제ㆍ산업_전망_2023-02(2)__2024년_경제산업_전망_13대_주력산업편.pdf'

#### PDF 파일 Page 별 Split

In [137]:
# pdf 파일 페이지별 split 함수
def chunk_pdf_with_pdfplumber(file_path, start_page, end_page):
    chunks = []
    
    with pdfplumber.open(file_path) as pdf:
        # 주어진 페이지 범위에 해당하는 페이지들을 처리합니다.
        for page_num in range(start_page - 1, end_page):
            page = pdf.pages[page_num]
            text = page.extract_text()
            if text:
                # 이스케이프 문자와 '<숫자-숫자>' 형태의 문구 제거
                cleaned_text = re.sub(r'\n|\r|\t', ' ', text)  # 이스케이프 문자 제거
                cleaned_text = re.sub(r'표<\d+-\d+>', '', cleaned_text)  # '<숫자-숫자>' 형태 제거
                cleaned_text = re.sub(r'<(그림|표) \d+-\d+>', '', cleaned_text)  # '<그림 숫자-숫자>', '<표 숫자-숫자>' 형태 제거
                chunks.append(cleaned_text.strip())
                

    return chunks

# 특정 페이지 범위 설정 (예: 13페이지부터 뒤에서 4페이지까지)
file_path = "경제산업동향_2024_11_2r_(2).pdf"
start_page = 3
end_page = -2  # 뒤에서 4페이지까지 의미

# PDF 페이지 수를 파악해 범위를 계산
with pdfplumber.open(file_path) as pdf:
    total_pages = len(pdf.pages)
    if end_page < 0:
        end_page = total_pages + end_page + 1  # 뒤에서부터 페이지 수 계산

# split 한 페이지 텍스트 document 형식으로 변환
chunks = [Document(page_content=doc) for doc in chunk_pdf_with_pdfplumber(file_path, start_page, end_page)]

In [138]:
chunks

[Document(page_content='< 요약 > □ 해외경제 : 日 3분기 성장률 전기비 0.2%, 유로존 성장률 0.4% 기록 ○ 미국은 10월 실물지표들의 혼조세 속에 트럼프 2기 행정부의 ‘관세’ 및 ‘감 세’ 추진이 예상되는 가운데 인플레 재심화, 경기 연착륙 난항 등 우려 부상 ○ 일본은 9월 실물지표들이 혼조세를 보인 가운데 3분기중 소비 증가에도 투자 부진 영향으로 성장률이 0.2%에 그친 반면, 유로존은 9월 실물지표들의 부진 에도 3분기 주요국들의 성장률 반등에 힘입어 역내 성장률이 0.4%로 상승 ○ 중국은 10월 소비와 수출 등 내·외수 지표들의 증가세가 전월보다 확대된 가 운데 올해 성장률 목표치 달성을 위한 中 정부의 전방위 경기부양책에 관심 ○ 달러화는 11월 들어 단기 상승에 따른 조정 심리 영향에도 트럼프 2기 출범 이후 재정적자 확대 우려와 연준의 소극적 금리 인하 전망으로 추가 상승세 ○ 유가는 중국 경기부양책 실망감과 미국 원유재고 증가 등으로 약세를 보인 이후, 러시아-우크라이나 확전 가능성 등 지정학적 불확실성 우려로 반등 □ 국내경기 : 9월 전산업생산 전월비 －0.3%, 소비 감소·투자 확대 ○ 지난 9월 전산업생산은 광공업생산이 소폭 감소 전환(전월비 –0.2%)하고, 서 비스업생산도 동반 감소(-0.7%)함에 따라 전월비 기준 재차 감소(-0.3%) - 소매판매는 비내구재·준내구재 판매의 동반 감소 여파로 전월비 0.4% 감 소한 반면, 설비투자는 기계류의 대폭 증가에 힘입어 8.4% 증가 □ 국내금융 : 10월 기업대출 확대 및 가계대출 증가세 둔화, 11월 중순 (11월 12일 ~ 25일) 금리 하락 및 원/달러 약보합 ○ 10월 국내 은행들의 기업대출이 대기업과 중소기업 대출이 모두 확대되면서 전월비 증가 규모가 확대된 반면, 가계대출(모기지론 양도 포함 시)은 주택담 보대출의 전월비 증가 규모가 축소되면서 전체 전월비 증가세가 더욱 둔화 - 1 -'),
 Document(page_content='○ 국

#### Local FAISS에 Chunk 저장

In [139]:
from langchain.embeddings import OpenAIEmbeddings

# 벡터 db 생성
vectorstore = FAISS.from_documents(documents=chunks, embedding=OpenAIEmbeddings())

# 로컬에 db 저장
vectorstore.save_local('./db/faiss')